In [3]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid

In [4]:
box_N = 128
box_L = 100.0
box_dx = box_L / box_N
def verify_box_spec(sim_path):
	config_path = os.path.join(sim_path, "simConfig.mat")
	simConfig = spio.loadmat(config_path)["simConfig"]
	if box_L != float(simConfig["Lbox"][0, 0][0, 0]):
		raise Exception(ValueError())
	if box_N != int(simConfig["N"][0, 0][0, 0]):
		raise Exception(ValueError())


VOL_SCALE_FACTOR = 4
vol_grid_axis = np.linspace((-box_L + box_dx) / 2, (box_L - box_dx) / 2, num=box_N//VOL_SCALE_FACTOR)
(GX, GY, GZ) = [gi.flatten() for gi in np.meshgrid(vol_grid_axis, vol_grid_axis, vol_grid_axis, indexing='ij')]
	

In [5]:
def load_snaps_at(sim_path, progress_indices):
	snap_paths = get_snap_paths(sim_path)
	outs = []
	for pr in progress_indices:
		ind = int(pr * (len(snap_paths) - 1))
		snap_path = snap_paths[ind]
		outs.append(load_Psi(snap_path))
	return outs


In [6]:
def load_Psi(snap_path):
	print(f"loading {snap_path}")
	snap = spio.loadmat(snap_path)
	Psi = np.stack([snap["Psi"][0, i] for i in range(3)], axis=0)
	print(f"loaded")
	return Psi

def get_snap_paths(sim_path):
	verify_box_spec(sim_path)
	sim_files = os.listdir(sim_path)
	searcher = r"snap-Psi-(\d+)-(\d+\.\d*)\.mat"
	snap_files = [re.search(searcher, fname) for fname in sim_files]
	snap_files = [(int(f.groups()[0]), float(f.groups()[1])) for f in snap_files if not(f is None)]
	snap_files = sorted(snap_files)
	snap_files = [f"snap-Psi-{iter}-{time}.mat" for (iter, time) in snap_files]
	snap_paths = [os.path.join(sim_path, f) for f in snap_files]
	return snap_paths


In [7]:
def plot_snap_density(Psi):
	Rho = get_density(Psi)
    
	Val = downscale_3d(Rho, VOL_SCALE_FACTOR)
	plot = pgo.Volume(
		x = GX,
		y = GY,
		z = GZ,
		value = np.log1p(Val).flatten() / 3,
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)
	return plot

In [8]:

kw_sq_nz = get_kw_square_nonzero_grid(box_N, box_dx)

def get_normalized_grav_potential(Rho):
	V_grav = np.fft.fftn(-Rho)
	V_grav = V_grav / kw_sq_nz
	V_grav = np.real(np.fft.ifftn(V_grav))
	min_V = np.min(V_grav)
	max_V = np.max(V_grav)
	return (V_grav - min_V) / (max_V - min_V)

def plot_snap_V_grav(Psi):
	Rho = get_density(Psi)
	V_grav = get_normalized_grav_potential(Rho)
	return pgo.Volume(
		x = GX, y = GY, z = GZ,
		value = 1 - downscale_3d(V_grav, VOL_SCALE_FACTOR).flatten(),
		opacity = 0.02,
		surface_count = 128,
		coloraxis = "coloraxis"
	)

In [9]:
def plot_snap_spins(Psi):
	Rho = get_density(Psi)
	Spins = get_spins(Psi)
	rt3 = 3**0.5
	Spp = [downscale_3d(Spin / Rho, VOL_SCALE_FACTOR).flatten() / rt3 for Spin in Spins]
	return pgo.Cone(
		x=GX, y=GY, z=GZ,
		u=Spp[0], v=Spp[1], w=Spp[2],
		coloraxis = "coloraxis"
	)

In [ ]:
Psis = load_snaps_at(f"../outputs/2022-07-30/8-solitons-random-128-attractive-run-1/", [0.0, 0.5, 1.0])
plots = [
	[plot_snap_density(Psi) for Psi in Psis],
	[plot_snap_V_grav(Psi) for Psi in Psis],
	[plot_snap_spins(Psi) for Psi in Psis]
]
fig, js = plot_multiple_3Ds(plots)
fig.update_layout(
	coloraxis = {
		'colorscale': 'jet',
		'cmin': 0.0,
		'cmax': 1.0,
	},
)
fig.write_html("plot_Rho_Spin_2022-07-30-run-1.html", post_script=js)

In [11]:
NUM_SIMULATIONS = 10
PROGRESS_STEPS = [1.0]
INTR_MODES = [
	# filename,		 label,				 color
	("nosi",		"No SI",			"#1abc9c"),
	("attractive",	"Attractive SI",	"#3498db"),
	("repulsive",	"Repulsive SI",		"#f39c12"),
]
all_Psis = [[load_snaps_at(f"../out_remote/2022-07-30/8-solitons-random-128-{filename}-run-{i}/", PROGRESS_STEPS) for (filename, _, _) in INTR_MODES] for i in range(1, 1+NUM_SIMULATIONS)]

loading ../out_remote/2022-07-30/8-solitons-random-128-nosi-run-1/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-attractive-run-1/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-repulsive-run-1/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-nosi-run-2/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-attractive-run-2/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-repulsive-run-2/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-nosi-run-3/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-attractive-run-3/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-repulsive-run-3/snap-Psi-12000-6367.27.mat
loaded
loading ../out_remote/2022-07-30/8-solitons-random-128-nosi-run-4/s

In [ ]:
# all_Psis[sim number][interaction mode][progress step][component]

In [12]:
from plotly.subplots import make_subplots
from tqdm import tqdm
def make_plots(plot_func):
	fig = make_subplots(NUM_SIMULATIONS, len(PROGRESS_STEPS), shared_xaxes=True, shared_yaxes=True)
	for (simul, sim_Psis) in tqdm(enumerate(all_Psis, start=1)):
		for (intr_idx, sim_intr_Psis) in enumerate(sim_Psis):
			for (pro, step_Psi) in enumerate(sim_intr_Psis, start=1):
				trace = plot_func(step_Psi)
				(_, label, color) = INTR_MODES[intr_idx]
				trace.name = label
				trace.line.color = color
				fig.add_trace(trace, simul, pro)

	fig = pgo.Figure(fig)
	return fig

NUM_BINS = 192

In [28]:
def plot_Rho_over_AvgRho_low(Psi):
	Rho = get_density(Psi)
	Values = Rho / np.average(Rho)
	counts, edges = np.histogram(Values, bins = NUM_BINS, range = (0.0, 1.0), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")
def plot_Rho_over_AvgRho_high(Psi):
	Rho = get_density(Psi)
	Values = Rho / np.average(Rho)
	max_value = np.max(Values)
	counts, edges = np.histogram(Values, bins = np.logspace(0, np.log10(max_value), num = NUM_BINS, base=10.0), range = (1.0, max_value), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Rho_over_AvgRho_low)
fig.update_yaxes(type = "log")
fig.write_html("2022-07-30_Rho_over_AvgRho_low.html")
fig = make_plots(plot_Rho_over_AvgRho_high)
fig.update_yaxes(type = "log")
fig.update_xaxes(type = "log")
fig.write_html("2022-07-30_Rho_over_AvgRho_high.html")

8it [00:10,  1.33s/it]
8it [00:15,  1.94s/it]


In [11]:
def plot_Spin_over_Rho(Psi):
	Rho = get_density(Psi)
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / Rho
	counts, edges = np.histogram(Values, bins = NUM_BINS, density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Spin_over_Rho)
fig.write_html("2022-07-30_Spin_over_Rho.html")

7it [01:39, 14.23s/it]


In [11]:

def plot_Spin_over_AvgSpin_low(Psi):
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / np.average(Norms)
	counts, edges = np.histogram(Values, bins = NUM_BINS, range = (0.0, 1.0), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")
def plot_Spin_over_AvgSpin_high(Psi):
	Spins = get_spins(Psi)
	Norms = np.sqrt(np.sum(np.square(Spins), axis=0))
	Values = Norms / np.average(Norms)
	max_value = np.max(Values)
	counts, edges = np.histogram(Values, bins = np.logspace(0, np.log10(max_value), num = NUM_BINS, base=10.0), range = (1.0, max_value), density = True)
	return pgo.Scatter(x=edges, y=counts, mode="lines+markers")

fig = make_plots(plot_Spin_over_AvgSpin_low)
fig.update_yaxes(type = "log")
fig.write_html("2022-07-30_Spin_over_AvgSpin_low.html")
fig = make_plots(plot_Spin_over_AvgSpin_high)
fig.update_yaxes(type = "log")
fig.update_xaxes(type = "log")
fig.write_html("2022-07-30_Spin_over_AvgSpin_high.html")

5it [00:42,  8.56s/it]
5it [00:58, 11.68s/it]


In [13]:
MOVING_AVERAGE_WINDOW = 128
Space3 = get_space_grid(box_N, box_dx)
def find_dense_circle(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]
    target_density = Rho_s[0] * 0.01
    centered_Rho_mavg = np.convolve(Rho_s, np.ones(
        MOVING_AVERAGE_WINDOW), mode='valid') / MOVING_AVERAGE_WINDOW
    boundary_arg = np.argmax(
        centered_Rho_mavg < target_density) + MOVING_AVERAGE_WINDOW
    boundary_range = slice(boundary_arg - MOVING_AVERAGE_WINDOW, boundary_arg)
    selected_range = slice(0, boundary_arg)
    boundary_radius = radius_s[boundary_arg]
    print(f"selection radius = {boundary_radius}")
    if boundary_arg < MOVING_AVERAGE_WINDOW * 2:
        print(f"warn: selected volume is small. boundary_arg={boundary_arg}")
    space_sorted = [sp.flatten()[sorter] for sp in Space3]
    boundary_positions = tuple([s[boundary_range] for s in space_sorted])
    selected_Psi = np.stack(
        [ps.flatten()[sorter][selected_range] for ps in Psi3], axis=0)
    return boundary_radius, boundary_positions, selected_Psi


In [14]:
Psi = all_Psis[6][0][-1]

In [18]:
def plot_rho_over_r(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]
    counts_we, edges = np.histogram(
        radius_s, weights=Rho_s, bins=np.linspace(1, 200, 400))
    counts_uw, _ = np.histogram(radius_s, bins=np.linspace(1, 200, 400))
    y = counts_we/np.maximum(counts_uw, 1)
    x = edges
    # cond = radius_s < 8
    # x = radius_s[cond]
    # y = Rho_s[cond]
    return pgo.Scatter(x=x, y=y, mode='markers')
fig = pgo.Figure(data=plot_rho_over_r(Psi))
fig.update_yaxes(type="log")
fig.update_xaxes(type="log")
pass
fig.write_html("2022-07-30_Rho_over_r.html")


In [ ]:
fig.show()

In [16]:
radius, boundary_points, selected_Psi = find_dense_circle(Psi)
(bx, by, bz) = boundary_points
plots = [
	[plot_snap_density(Psi)],
]
fig, js = plot_multiple_3Ds(plots)
fig.add_trace(pgo.Mesh3d(
	x=bx, y=by, z=bz,
	opacity=0.4, color='red', alphahull=0,
), 1, 1)
fig.update_layout(
	coloraxis = {
		'colorscale': 'viridis',
		'cmin': 0.0,
		'cmax': 1.0,
	},
)
fig.write_html("2022-07-30-core.html", post_script=js)
pass

selection radius = 3.983608994994363


In [159]:
def spin_norm_over_mass(Psi):
	norm = np.sqrt(np.sum(np.square(
		np.sum(get_spins(Psi), axis=tuple(range(1,Psi.ndim)))
	)))
	mass = np.sum(get_density(Psi))
	return norm / mass

def test_spin_norm_over_mass(Psis):
	x = []
	y = []
	hovertexts = []
	for (n, Psi) in enumerate(Psis, start=1):
		radius, boundary_points, selected_Psi = find_dense_circle(Psi)
		x.append(spin_norm_over_mass(Psi))
		y.append(spin_norm_over_mass(selected_Psi))
		hovertexts.append(f"simulation {n} selected_radius={radius}")
	return pgo.Scatter(
		x=x, y=y,
		hovertext=hovertexts,
		mode="markers"
	)


In [160]:
fig = make_subplots(1, 1)
for (mode_idx, (_, label, color)) in enumerate(INTR_MODES):
	trace = test_spin_norm_over_mass([simuls[mode_idx][-1] for simuls in all_Psis])
	trace.marker.color = color	
	trace.name = label
	fig.add_trace(trace, 1, 1)
fig = pgo.Figure(fig)
fig.update_layout(yaxis_range=[0.0, 1.0], xaxis_range=[0.0, 1.0])
fig.write_html("2022-07-30-total-vs-core-37.html")

densest_point_idx: (65, 68, 107)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (94, 10, 105)
selection radius = 2.5816687103551166
warn: selected volume is small. boundary_arg=17
densest_point_idx: (7, 82, 19)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (98, 13, 64)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (113, 71, 85)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (48, 0, 113)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (10, 37, 106)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (50, 69, 64)
selection radius = 2.329165268936858
warn: selected volume is small. boundary_arg=0
densest_point_idx: (65, 76, 105)
selection radius = 2.3291652689368

In [94]:
print(selected_Psi.shape)
print(np.sum(get_spins(selected_Psi), axis=1).shape)
print(np.sum(get_spins(centered_Psi), axis=(1, 2, 3)).shape)

(3, 675)
(3,)
(3,)


In [99]:
tuple(range(1,2))

(1,)

In [154]:
print(spin_norm_over_mass(Psi))
print(spin_norm_over_mass(selected_Psi))
print(np.average(get_density(Psi)))
print(np.average(get_density(selected_Psi)))


0.2494654548115159
0.6278263330779291
0.07522553430817373
77.22379585087349


In [128]:
sim_paths = [f"../outputs/2022-07-30/8-solitons-random-128-nosi-run-{i}/" for i in range(1, 1+NUM_SIMULATIONS)]
configs = []
for sim_path in sim_paths:
	config = spio.loadmat(os.path.join(sim_path, "simConfig.mat"))
	configs.append(config)

In [138]:
sizes = [config['simConfig']['sizes'][0][0] for config in configs]

In [141]:
[np.max(s) for s in sizes]

[3.951379331054033,
 3.9239189007638346,
 3.779040647205886,
 3.840470075396803,
 3.8839269483701537,
 3.7529775798556253,
 3.7325316943822875,
 3.7450558879578937]